### bd econ CPS price level retrieval

bd_CPS_cpi.ipynb

January 30, 2019

Brian Dew, @bd_econ

-----
This file is used to download the 1989-present consumer price indices for all urban consumers, as well as for urban consumers in the four census regions: Northeast, Midwest, South, and West. These data are used to adjust the wage and overtime values reported in the current population survey for changes to the price level. The 1989 to 1999 consumer price data does tend to overstate the inflation rate, thus making wage growth seem lower than it actually was over the period. Therefore, I may opt, at a later date, to replace the regional CPI approach with the BLS research series, referred to as the CPI-U-RS.

To run this code, you need an API key from the [Bureau of Labor Statistics](https://www.bls.gov/developers/home.htm) (free), saved in a python file called `config.py` in the same directory as this notebook. The python file can be created from a blank text file that contains: `bls_key = API_key_pasted_here`.

In [1]:
# Import packages and bls api key
import pandas as pd
print('pandas:', pd.__version__)
import os
import requests
print('requests:', requests.__version__)
import json
import config

os.chdir('/home/brian/Documents/CPS/data/clean/')

pandas: 0.24.2
requests: 2.22.0


##### BLS API retrieval

Run after BLS releases CPI data for period of interest.

In [2]:
# Code to update CPI as needed
api_url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

# API key in config.py which contains: bls_key = 'key'
key = f'?registrationkey={config.bls_key}'

# Series stored as a dictionary
series_dict = {'CUUR0000SA0': 'ALL',
               'CUUR0100SA0': 'Northeast',
               'CUUR0200SA0': 'Midwest',
               'CUUR0300SA0': 'South',
               'CUUR0400SA0': 'West'}

# Start year and end year
date_r = (1989, 2019)

# Because API requests are limited to 10 years at a time,
# this code block splits the dates above in 10-year chunks
dates = [(str(date_r[0]), str(date_r[1]))]
while int(dates[-1][1]) - int(dates[-1][0]) > 10:
    dates = [(str(date_r[0]), str(date_r[0]+9))]
    d1 = int(dates[-1][0])
    while int(dates[-1][1]) < date_r[1]:
        d1 = d1 + 10
        d2 = min([date_r[1], d1+9])
        dates.append((str(d1),(d2)))
        
df = pd.DataFrame()

for start, end in dates:
    # Submit the list of series as data
    data = json.dumps({
        "seriesid": list(series_dict.keys()),
        "startyear": start, "endyear": end})

    # Post request for the data
    p = requests.post(f'{api_url}{key}', 
        headers={'Content-type': 'application/json'}, 
        data=data).json()
    for s in p['Results']['series']:
        col = series_dict[s['seriesID']]
        for r in s['data']:
            date = pd.to_datetime(f'{r["periodName"]} {r["year"]}')
            df.at[date, col] = float(r['value'])
df = df.sort_index()
# Output results
print(f'Post Request Status: {p["status"]}')
print('Latest month: ', df.index[-1])
      
cpi = (df.iloc[-1] / df)
cpi.to_csv('cpi.csv')

Post Request Status: REQUEST_SUCCEEDED
Latest month:  2019-10-01 00:00:00
